<a href="https://colab.research.google.com/github/psy794/stock_pjtt/blob/master/XGBoost(0_8769)_0724_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#구글드라이브를 mount해요

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle

#데이터 프레임을 불러옵니다

In [ ]:
data = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/20230723dataframe_일부drop.pkl', 'rb'))
data

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,usdtocny,...,팔라듐,구리,알루미늄,아연,니켈,주석,난방유,천연가스,가솔린RBOB,WTI
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310,6.3690,...,2856.7,4.7,2461.5,3043.0,18147,33366,2.0715,3.104,2.1704,67.72
1,2021-06-01,5860,5940,5750,5780,63836,371440050,-2.360000,095570,6.3690,...,2856.7,4.7,2461.5,3043.0,18147,33366,2.0715,3.104,2.1704,67.72
2,2021-06-01,35500,35600,34150,34400,103691,3580613850,-2.820000,006840,6.3690,...,2856.7,4.7,2461.5,3043.0,18147,33366,2.0715,3.104,2.1704,67.72
3,2021-06-01,14600,14950,13800,14950,462544,6605258500,2.400000,054620,6.3690,...,2856.7,4.7,2461.5,3043.0,18147,33366,2.0715,3.104,2.1704,67.72
4,2021-06-01,29150,29150,28800,29050,131987,3831133400,-0.170000,265520,6.3690,...,2856.7,4.7,2461.5,3043.0,18147,33366,2.0715,3.104,2.1704,67.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929995,2023-05-30,3005,3035,2955,2980,272284,815753095,-0.670000,189980,7.0723,...,1378.6,3.7,2230.0,2323.5,21235,25705,2.2808,2.327,2.5959,69.46
929996,2023-05-30,3250,3255,3195,3215,50218,161528300,-0.620000,000540,7.0723,...,1378.6,3.7,2230.0,2323.5,21235,25705,2.2808,2.327,2.5959,69.46
929997,2023-05-30,1344,1395,1340,1370,130664,177771369,1.780000,003280,7.0723,...,1378.6,3.7,2230.0,2323.5,21235,25705,2.2808,2.327,2.5959,69.46
929998,2023-05-30,9170,9260,9170,9200,141932,1304575190,0.000000,037440,7.0723,...,1378.6,3.7,2230.0,2323.5,21235,25705,2.2808,2.327,2.5959,69.46


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/separated_datasets_sectors.pkl', 'rb') as file:
    separated_datasets_sectors = pickle.load(file)

In [ ]:
data.columns

Index(['날짜', '시가', '고가', '저가', '종가', '거래량', '거래대금', '등락률', '종목코드', 'usdtocny',
       'eurtousd', 'msci_emerging', 'usdtoaud', 'usdtojpy', 'usdtokrw', 'dow',
       'snp500', 'nasdaq', 'semicon_index', 'us10y_tsy', '시가총액', '금', '은',
       '백금', '팔라듐', '구리', '알루미늄', '아연', '니켈', '주석', '난방유', '천연가스', '가솔린RBOB',
       'WTI'],
      dtype='object')

In [ ]:
separated_datasets_sectors

In [ ]:
sector_dfs = []

for sector_name, sector_df in separated_datasets_sectors.items():
  sector_df['업종명'] = sector_name
  sector_dfs.append(sector_df)

merged_data = pd.concat(sector_dfs)


merged_data

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,Close_mv5,...,nasdaq,semicon_index,euro50,us10y_tsy,fast_k,fast_d,시가총액,slow_k,slow_d,업종명
55328,2021-06-01,20400,20800,20100,20550,211150,4320781100,1.23,060720,NaN,...,13736.48,3174.32,4071.75,1.615,64.285714,64.285714,486498706650,64.285714,64.285714,IT부품
55329,2021-06-02,20950,21150,20700,20800,344482,7202011200,1.22,060720,NaN,...,13756.33,3196.98,4088.50,1.591,66.666667,65.714286,492417182400,65.714286,65.142857,IT부품
55330,2021-06-03,21000,21700,21000,21650,434569,9314752200,4.09,060720,NaN,...,13614.51,3139.19,4079.24,1.625,96.875000,80.474624,512539999950,80.474624,72.405273,IT부품
55331,2021-06-04,21650,22100,21350,21750,333074,7254783050,0.46,060720,NaN,...,13814.49,3214.14,4089.38,1.560,82.500000,81.315934,514907390250,81.315934,76.106625,IT부품
55332,2021-06-07,21850,21900,21050,21250,241745,5136611800,-2.30,060720,21200.0,...,13881.72,3191.66,4097.65,1.569,57.500000,72.173324,503070438750,72.173324,74.596685,IT부품
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981079,2023-05-23,4780,4805,4740,4770,25435,121248710,-0.21,079980,4723.0,...,12560.25,3177.44,4342.38,3.698,79.032258,67.391066,164565000000,67.391066,51.804965,화학
981080,2023-05-24,4770,4770,4695,4770,21634,102170545,0.00,079980,4753.0,...,12484.16,3124.09,4263.74,3.719,79.032258,71.271463,164565000000,71.271463,58.293798,화학
981081,2023-05-25,4730,4765,4680,4750,30459,143268485,-0.42,079980,4771.0,...,12698.09,3336.72,4269.64,3.814,72.580645,71.707857,163875000000,71.707857,62.765151,화학
981082,2023-05-26,4750,4750,4640,4695,28337,132163680,-1.16,079980,4753.0,...,12975.69,3545.67,4337.50,3.810,54.838710,66.084808,161977500000,66.084808,63.871703,화학


In [ ]:
exclude_columns = ['업종명','종목코드']
drop_columns = [col for col in merged_data.columns if col not in exclude_columns]
drop_columns

['날짜',
 '시가',
 '고가',
 '저가',
 '종가',
 '거래량',
 '거래대금',
 '등락률',
 'Close_mv5',
 'Close_mv10',
 'Close_mv20',
 'OBV',
 'OBV_EMA',
 'sse_composite_index',
 'csi300_index',
 'usdtocny',
 'eurtousd',
 'msci_emerging',
 'usdtoaud',
 'usdtojpy',
 'usdtokrw',
 'dow',
 'snp500',
 'nasdaq',
 'semicon_index',
 'euro50',
 'us10y_tsy',
 'fast_k',
 'fast_d',
 '시가총액',
 'slow_k',
 'slow_d']

In [ ]:
merged_data = merged_data.drop(columns=drop_columns)
merged_data

,종목코드,업종명
55328,060720,IT부품
55329,060720,IT부품
55330,060720,IT부품
55331,060720,IT부품
55332,060720,IT부품
...,...,...
981079,079980,화학
981080,079980,화학
981081,079980,화학
981082,079980,화학


In [ ]:
merged_data.drop_duplicates(subset=['종목코드','업종명'], keep= 'first', inplace = True)
merged_data

,종목코드,업종명
55328,060720,IT부품
81510,091340,IT부품
83980,151910,IT부품
152152,190510,IT부품
155610,267320,IT부품
...,...,...
973674,298050,화학
974168,298020,화학
974662,298000,화학
975156,093370,화학


In [ ]:
# 결과를 데이터프레임을 CSV 파일로 저장
merged_data.to_csv('/content/drive/MyDrive/Colab Notebooks/종목코드별업종.csv', index=False)

In [ ]:
# data에 업종 병합
data = data.merge(merged_data[['업종명','종목코드']], on='종목코드', how='left')
data

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,usdtocny,...,구리,알루미늄,아연,니켈,주석,난방유,천연가스,가솔린RBOB,WTI,업종명
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310,6.3690,...,4.7,2461.5,3043.0,18147,33366,2.0715,3.104,2.1704,67.72,기계·장비
1,2021-06-01,5860,5940,5750,5780,63836,371440050,-2.360000,095570,6.3690,...,4.7,2461.5,3043.0,18147,33366,2.0715,3.104,2.1704,67.72,서비스업
2,2021-06-01,35500,35600,34150,34400,103691,3580613850,-2.820000,006840,6.3690,...,4.7,2461.5,3043.0,18147,33366,2.0715,3.104,2.1704,67.72,기타금융
3,2021-06-01,14600,14950,13800,14950,462544,6605258500,2.400000,054620,6.3690,...,4.7,2461.5,3043.0,18147,33366,2.0715,3.104,2.1704,67.72,금융
4,2021-06-01,29150,29150,28800,29050,131987,3831133400,-0.170000,265520,6.3690,...,4.7,2461.5,3043.0,18147,33366,2.0715,3.104,2.1704,67.72,반도체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929995,2023-05-30,3005,3035,2955,2980,272284,815753095,-0.670000,189980,7.0723,...,3.7,2230.0,2323.5,21235,25705,2.2808,2.327,2.5959,69.46,음식료·담배
929996,2023-05-30,3250,3255,3195,3215,50218,161528300,-0.620000,000540,7.0723,...,3.7,2230.0,2323.5,21235,25705,2.2808,2.327,2.5959,69.46,보험
929997,2023-05-30,1344,1395,1340,1370,130664,177771369,1.780000,003280,7.0723,...,3.7,2230.0,2323.5,21235,25705,2.2808,2.327,2.5959,69.46,운수창고업
929998,2023-05-30,9170,9260,9170,9200,141932,1304575190,0.000000,037440,7.0723,...,3.7,2230.0,2323.5,21235,25705,2.2808,2.327,2.5959,69.46,기타서비스


In [ ]:
data.dtypes

날짜               datetime64[ns]
시가                        int32
고가                        int32
저가                        int32
종가                        int32
거래량                       int32
거래대금                      int64
등락률                     float32
종목코드                     object
usdtocny                float64
eurtousd                float64
msci_emerging           float64
usdtoaud                float64
usdtojpy                float64
usdtokrw                float64
dow                     float64
snp500                  float64
nasdaq                  float64
semicon_index           float64
us10y_tsy               float64
시가총액                      int64
금                       float64
은                       float64
백금                      float64
팔라듐                     float64
구리                      float64
알루미늄                    float64
아연                      float64
니켈                        int64
주석                        int64
난방유                     float64
천연가스    

In [ ]:
data.columns

Index(['날짜', '시가', '고가', '저가', '종가', '거래량', '거래대금', '등락률', '종목코드', 'usdtocny',
       'eurtousd', 'msci_emerging', 'usdtoaud', 'usdtojpy', 'usdtokrw', 'dow',
       'snp500', 'nasdaq', 'semicon_index', 'us10y_tsy', '시가총액', '금', '은',
       '백금', '팔라듐', '구리', '알루미늄', '아연', '니켈', '주석', '난방유', '천연가스', '가솔린RBOB',
       'WTI', '업종명'],
      dtype='object')

In [ ]:
# 업종명을 기준으로 데이터프레임을 그룹화합니다.
grouped_data_sectors = data.groupby('업종명')

# 그룹화된 데이터프레임을 업종명별로 따로 저장합니다.
separated_datasets_sectors = {sector: group for sector, group in grouped_data_sectors}

In [ ]:
# 업종명을 기준으로 데이터프레임을 그룹화합니다.
grouped_data_sectors = data.groupby('업종명')

# 그룹화된 데이터프레임을 업종명별로 따로 저장합니다.
separated_datasets_sectors = {sector: group for sector, group in grouped_data_sectors}

In [ ]:
dict_data = separated_datasets_sectors

In [ ]:
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np

industries = list(dict_data.keys())  # 전체 업종 선택
window_size = 3  # 윈도우 사이즈를 2로 통일
sectors_features = {}  # 각 업종별 최종 선택된 피처들을 저장할 딕셔너리

for industry_index, industry in enumerate(industries):
    print(f"Processing industry: {industry}")
    codes = dict_data[industry]['종목코드'].unique()

    feature_importances = {}  # 각 피처의 중요도를 저장할 딕셔너리

    # 첫번째 단계: 각 종목코드에 대해 모델을 학습하고 피처 중요도를 평균내기
    for code_index, code in enumerate(codes):
        print(f"Processing code: {code}")
        df = dict_data[industry][dict_data[industry]['종목코드'] == code].copy()

        if len(df) < window_size:  # 데이터가 충분하지 않은 경우 건너뜁니다.
            print(f"Skipping code {code} due to insufficient data")
            continue

        df['날짜'] = pd.to_datetime(df['날짜'])
        df.set_index('날짜', inplace=True)

        selected_vars = df[['종가','usdtocny','eurtousd', 'msci_emerging', 'usdtoaud', 'usdtojpy', 'usdtokrw', 'dow','snp500', 'nasdaq', 'semicon_index', 'us10y_tsy',
                            '시가총액', '금', '은', '백금', '팔라듐', '구리', '알루미늄', '아연', '니켈', '주석', '난방유', '천연가스', '가솔린RBOB', 'WTI',]]

        # 윈도우 사이즈와 롤링 윈도우 적용
        moving_vars = selected_vars.rolling(window=window_size).mean().shift(1)
        moving_vars['종가'] = selected_vars['종가']

        moving_vars = moving_vars.dropna()

        train = moving_vars[moving_vars.index < '2023-05-01']

        # 학습 데이터셋이 충분하지 않은 경우 건너뜁니다.
        if train.empty:
            print(f"Skipping code {code} due to insufficient train data")
            continue

        test = moving_vars[(moving_vars.index >= '2023-05-01') & (moving_vars.index <= '2023-05-30')]

        # 테스트 데이터셋이 비어있지 않은 경우에만 모델 학습 및 예측을 수행
        if not test.empty:
            X_train = train.drop('종가', axis=1)
            y_train = train['종가']

            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)

            model = xgb.XGBRegressor(objective='reg:squarederror')

            print(f'Training model for code {code}...')
            model.fit(X_train_scaled, y_train)

            # 피처 중요도를 저장
            for i, col in enumerate(X_train.columns):
                if col not in feature_importances:
                    feature_importances[col] = []
                feature_importances[col].append(model.feature_importances_[i])

    # 피처 중요도의 평균을 구하고, 중요도가 0.1 이상인 피처만 선택
    avg_importances = {k: np.mean(v) for k, v in feature_importances.items()}
    selected_features = [k for k, v in avg_importances.items() if v > 0.01]

    # 최종 학습에 사용된 피처들을 딕셔너리에 저장
    sectors_features[industry] = selected_features

    print(f'Selected features for {industry}:')
    for feature in selected_features:
        print(feature)

print("\nFinal selected features for each industry:")
for industry, features in sectors_features.items():
    print(f"Industry: {industry}, Features: {features}")

Processing industry: IT부품
Processing code: 060720
Training model for code 060720...
Processing code: 091340
Training model for code 091340...
Processing code: 151910
Training model for code 151910...
Processing code: 190510
Training model for code 190510...
Processing code: 267320
Training model for code 267320...
Processing code: 085670
Training model for code 085670...
Processing code: 068240
Training model for code 068240...
Processing code: 078600
Training model for code 078600...
Processing code: 079960
Training model for code 079960...
Processing code: 032960
Training model for code 032960...
Processing code: 066670
Training model for code 066670...
Processing code: 110990
Training model for code 110990...
Processing code: 066900
Training model for code 066900...
Processing code: 290550
Training model for code 290550...
Processing code: 300120
Training model for code 300120...
Processing code: 096640
Training model for code 096640...
Processing code: 257370
Training model for cod

In [ ]:
# sectors_features 딕셔너리를 DataFrame으로 변환
df = pd.DataFrame(list(sectors_features.items()), columns=['Industry', 'Features'])


In [ ]:
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
import pandas as pd

industries = list(dict_data.keys())  # 전체 업종 선택
window_size = 1  # 윈도우 사이즈를 1로 설정

result_df = pd.DataFrame()  # 결과를 저장할 데이터프레임 초기화

# 각 업종에 대해 모델을 학습하고 테스트
for industry in industries:
    print(f"Processing industry: {industry}")
    codes = dict_data[industry]['종목코드'].unique()

    # 각 종목코드에 대해 모델을 학습하고 테스트
    for code in codes:
        print(f"Processing code: {code}")
        df = dict_data[industry][dict_data[industry]['종목코드'] == code].copy()

        df['날짜'] = pd.to_datetime(df['날짜'])
        df.set_index('날짜', inplace=True)

        selected_vars = df[['종가'] + sectors_features[industry]]  # 선택된 피처와 종가 사용

        # 윈도우 사이즈와 롤링 윈도우 적용
        moving_vars = selected_vars.rolling(window=window_size).mean().shift(1)

        # 예측 대상인 종가를 제외한 피처들만 사용
        moving_vars = moving_vars.drop('종가', axis=1)

        # 실제 종가를 추가
        moving_vars['종가'] = selected_vars['종가']

        moving_vars = moving_vars.dropna()

        train = moving_vars[(moving_vars.index >= '2021-06-01') & (moving_vars.index <= '2022-12-31')]                  #여기 잘못된거 다시 보면서 알았다...다시 돌려보고 이 파일 저장 후, train기간 수정하자!
        test = moving_vars[(moving_vars.index >= '2023-05-20') & (moving_vars.index <= '2023-05-29')] #테스트기간 5일 늘렸음

        # 테스트 데이터셋이 비어있지 않은 경우에만 모델 학습 및 예측을 수행
        if not test.empty:
            X_train = train.drop('종가', axis=1)
            y_train = train['종가']
            X_test = test.drop('종가', axis=1)
            y_test = test['종가']

            scaler = MinMaxScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            model = xgb.XGBRegressor(objective='reg:squarederror')

            print(f'Training model for code {code}...')
            model.fit(X_train_scaled, y_train)

            print(f'Predicting for code {code}...')
            predictions = model.predict(X_test_scaled)

            # 5월30일의 예측 종가와 5일간의 평균 종가 차이 계산
            average_close = y_test.mean()
            difference = average_close - predictions[-1]

            print(f"Code: {code}, Predicted Close on May 30: {predictions[-1]}, Average Close from May 20 to May 29: {average_close}, Difference: {difference}")

            # 결과를 데이터프레임에 추가
            result_row = pd.DataFrame({'종목코드': [code], 'Difference': [difference], '예측종가_5월30일': [predictions[-1]]})

            result_df = pd.concat([result_df, result_row])

# 결과를 데이터프레임을 CSV 파일로 저장
# result_df.to_csv('result.csv', index=False)
result_df

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Predicting for code 002150...
Code: 002150, Predicted Close on May 30: 8912.79296875, Average Close from May 20 to May 29: 8794.0, Difference: -118.79296875
Processing code: 102260
Training model for code 102260...
Predicting for code 102260...
Code: 102260, Predicted Close on May 30: 4583.046875, Average Close from May 20 to May 29: 4610.0, Difference: 26.953125
Processing code: 032350
Training model for code 032350...
Predicting for code 032350...
Code: 032350, Predicted Close on May 30: 11610.353515625, Average Close from May 20 to May 29: 11396.0, Difference: -214.353515625
Processing code: 286940
Training model for code 286940...
Predicting for code 286940...
Code: 286940, Predicted Close on May 30: 27634.056640625, Average Close from May 20 to May 29: 28040.0, Difference: 405.943359375
Processing code: 007120
Training model for code 007120...
Predicting for code 007120...
Code: 007120, Predicted Close on May 30: 1668.02490234375, Average Close 

,종목코드,Difference,예측종가_5월30일
0,060720,844.277344,19505.722656
0,091340,-56.130859,4233.130859
0,151910,-321.222559,866.622559
0,190510,378.109375,15771.890625
0,267320,68.497559,3399.502441
...,...,...,...
0,298050,3796.781250,450103.218750
0,298020,-12848.500000,407448.500000
0,298000,-7270.539062,108830.539062
0,093370,409.878906,12762.121094


In [ ]:
# 주가 대비 변동률 계산
result_df['주가대비변동률'] = result_df['Difference'] / result_df['예측종가_5월30일']


# 순위 매기기
result_df['순위'] = result_df['주가대비변동률'].rank(ascending=False)

In [ ]:
result_df

,종목코드,Difference,예측종가_5월30일,주가대비변동률,순위
0,060720,844.277344,19505.722656,0.043284,442.0
0,091340,-56.130859,4233.130859,-0.013260,1166.0
0,151910,-321.222559,866.622559,-0.370660,1961.0
0,190510,378.109375,15771.890625,0.023974,620.0
0,267320,68.497559,3399.502441,0.020149,661.0
...,...,...,...,...,...
0,298050,3796.781250,450103.218750,0.008435,830.0
0,298020,-12848.500000,407448.500000,-0.031534,1381.0
0,298000,-7270.539062,108830.539062,-0.066806,1631.0
0,093370,409.878906,12762.121094,0.032117,531.0


In [ ]:
#주가대비변동률이 음수이고 5일간 평균 종가와 예측 종가의 차이가 큰 마이너스 값을 하위200개에 배치하고자 함
# 주가대비변동률이 음수이고 5일간 평균 종가와 예측 종가의 차이가 큰 마이너스 값인 경우를 선택
filtered_df = result_df[(result_df['주가대비변동률'] < 0) & (result_df['Difference'] < 0)]

# 순위를 매기기 위해 'Difference' 컬럼을 기준으로 내림차순 정렬
filtered_df = filtered_df.sort_values('Difference', ascending=False)

# 순위 컬럼 추가 및 순위 매김
filtered_df['순위'] = filtered_df['Difference'].rank(ascending=False)

# 결과 확인
print(filtered_df[['종목코드', 'Difference', '순위']])

      종목코드    Difference      순위
0   001380     -0.550513     1.0
0   006980     -0.613281     2.0
0   042940     -0.937781     3.0
0   094970     -0.989746     4.0
0   008700     -1.132080     5.0
..     ...           ...     ...
0   017390 -46859.093750  1036.0
0   285130 -47194.171875  1037.0
0   003240 -48142.812500  1038.0
0   207940 -76812.875000  1039.0
0   051900 -80133.500000  1040.0

[1040 rows x 3 columns]


In [ ]:
# down 200에 하위 200개 저장
down_200 = filtered_df.tail(200)

# down 200의 순위를 다시 매김
down_200['Rank'] = range(1801, 2001)

# 최종 순위를 오름차순으로 정렬
final_ranking = pd.concat([filtered_df.head(len(filtered_df) - 200), down_200])

# 최종 순위를 하위 200개만 저장
final_ranking = final_ranking.tail(200)

# 결과 확인
final_ranking

<ipython-input-66-7460fe3fb49e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  down_200['Rank'] = range(1801, 2001)


,종목코드,Difference,예측종가_5월30일,주가대비변동률,순위,Rank
0,011790,-1433.812500,97233.812500,-0.014746,841.0,1801.0
0,109960,-1436.730957,3115.730957,-0.461122,842.0,1802.0
0,347860,-1443.378906,11979.378906,-0.120489,843.0,1803.0
0,194370,-1445.944336,13997.944336,-0.103297,844.0,1804.0
0,002020,-1446.044922,21726.044922,-0.066558,845.0,1805.0
...,...,...,...,...,...,...
0,017390,-46859.093750,134599.093750,-0.348138,1036.0,1996.0
0,285130,-47194.171875,121074.171875,-0.389796,1037.0,1997.0
0,003240,-48142.812500,716942.812500,-0.067150,1038.0,1998.0
0,207940,-76812.875000,862212.875000,-0.089088,1039.0,1999.0


In [ ]:
final_ranking['Rank'] = final_ranking['Rank'].astype(int)
final_ranking

,종목코드,Difference,예측종가_5월30일,주가대비변동률,순위,Rank
0,011790,-1433.812500,97233.812500,-0.014746,841.0,1801
0,109960,-1436.730957,3115.730957,-0.461122,842.0,1802
0,347860,-1443.378906,11979.378906,-0.120489,843.0,1803
0,194370,-1445.944336,13997.944336,-0.103297,844.0,1804
0,002020,-1446.044922,21726.044922,-0.066558,845.0,1805
...,...,...,...,...,...,...
0,017390,-46859.093750,134599.093750,-0.348138,1036.0,1996
0,285130,-47194.171875,121074.171875,-0.389796,1037.0,1997
0,003240,-48142.812500,716942.812500,-0.067150,1038.0,1998
0,207940,-76812.875000,862212.875000,-0.089088,1039.0,1999


In [ ]:

# Difference, 예측종가_5월30일, Rank, 주가대비변동률, 순위 컬럼 제거
final_ranking = final_ranking.drop(['Difference', '예측종가_5월30일','주가대비변동률', '순위'], axis=1)

final_ranking['종목코드'] = final_ranking['종목코드'].astype(str).str.zfill(6)
final_ranking['종목코드'] = np.where(final_ranking['종목코드'].str.len() == 6, 'A' + final_ranking['종목코드'], final_ranking['종목코드'])
final_ranking

,종목코드,Rank
0,A011790,1801
0,A109960,1802
0,A347860,1803
0,A194370,1804
0,A002020,1805
...,...,...
0,A017390,1996
0,A285130,1997
0,A003240,1998
0,A207940,1999


In [ ]:
final_ranking.rename(columns={'Rank': '순위'}, inplace=True)


In [ ]:
result_df

,종목코드,Difference,예측종가_5월30일,주가대비변동률,순위
0,060720,844.277344,19505.722656,0.043284,442.0
0,091340,-56.130859,4233.130859,-0.013260,1166.0
0,151910,-321.222559,866.622559,-0.370660,1961.0
0,190510,378.109375,15771.890625,0.023974,620.0
0,267320,68.497559,3399.502441,0.020149,661.0
...,...,...,...,...,...
0,298050,3796.781250,450103.218750,0.008435,830.0
0,298020,-12848.500000,407448.500000,-0.031534,1381.0
0,298000,-7270.539062,108830.539062,-0.066806,1631.0
0,093370,409.878906,12762.121094,0.032117,531.0


In [ ]:
result_df = result_df.reset_index(drop=True)


In [ ]:
# 주가대비변동률에 대한 조건 포함해서 5일간 평균 종가와 예측 종가 차이가 양수인것들 기준으로 순위매겨서 상위200개만 추출
# threshold를 적게 할수록 주가의 안정성이 높은 종목들이 상위권에 포함될 가능성이 높아짐

threshold = 0.1
result_df['rank'] = result_df[(result_df['주가대비변동률'] < threshold) & (result_df['Difference'] > 0)].rank(ascending=False)['Difference']


In [ ]:

# NaN 값을 가진 rank를 하위로 배치하고, rank를 오름차순으로 정렬
result_df['rank'] = result_df['rank'].fillna(result_df['rank'].max() + 1)
result_df['rank'] = result_df['rank'].astype(int)
result_df = result_df.sort_values(by='rank')
# 상위 200개 행만 유지하고 나머지 행은 제거
result_df = result_df.head(200)
result_df

,종목코드,Difference,예측종가_5월30일,주가대비변동률,순위,rank
1698,005490,28672.875000,339827.125000,0.084375,263.0,1
1847,051910,17826.750000,693573.250000,0.025703,604.0,2
1725,000670,17241.781250,523758.218750,0.032919,525.0,3
1432,066970,14376.671875,241923.328125,0.059427,351.0,4
776,036570,12781.343750,357218.656250,0.035780,503.0,5
...,...,...,...,...,...,...
1258,003960,565.152344,28544.847656,0.019799,667.0,196
505,090850,560.102539,5919.897461,0.094614,234.0,197
1925,101240,559.086426,7756.913574,0.072076,302.0,198
113,006360,551.425781,20958.574219,0.026310,596.0,199


In [ ]:
# Difference, 예측종가_5월30일, Rank, 주가대비변동률, 순위 컬럼 제거
result_df = result_df.drop(['Difference', '예측종가_5월30일', '주가대비변동률', '순위'], axis=1)
result_df['종목코드'] = result_df['종목코드'].astype(str).str.zfill(6)
result_df['종목코드'] = np.where(result_df['종목코드'].str.len() == 6, 'A' + result_df['종목코드'], result_df['종목코드'])
result_df.rename(columns={'rank': '순위'}, inplace=True)
# 결과 확인
result_df

,종목코드,순위
1698,A005490,1
1847,A051910,2
1725,A000670,3
1432,A066970,4
776,A036570,5
...,...,...
1258,A003960,196
505,A090850,197
1925,A101240,198
113,A006360,199


In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [ ]:
# final_ranking에서 종목코드와 순위 컬럼 선택
final_ranking_subset = final_ranking[['종목코드', '순위']]

# sample_submission에 final_ranking 순위 병합
sample_submission = pd.merge(sample_submission, final_ranking_subset, on='종목코드', how='left')

# 병합된 순위 컬럼명을 수정
sample_submission.rename(columns={'순위': '순위_final_ranking'}, inplace=True)

# result_df에서 종목코드와 순위 컬럼 선택
result_df_subset = result_df[['종목코드', '순위']]

# sample_submission에 result_df 순위 병합
sample_submission = pd.merge(sample_submission, result_df_subset, on='종목코드', how='left')

# 병합된 순위 컬럼명을 수정
sample_submission.rename(columns={'순위': '순위_result_df'}, inplace=True)

# '순위_y'와 '순위_result_df' 컬럼의 NaN을 0으로 채우기
sample_submission['순위_y'].fillna(0, inplace=True)
sample_submission['순위_result_df'].fillna(0, inplace=True)

# '순위_y'와 '순위_result_df' 컬럼을 더한 값을 '순위' 컬럼에 추가
sample_submission['순위'] = sample_submission['순위_y'] + sample_submission['순위_result_df']

# '순위_x', '순위_y', '순위_result_df' 컬럼 삭제
sample_submission.drop(['순위_x', '순위_y', '순위_result_df'], axis=1, inplace=True)

# '순위' 컬럼을 int형으로 변환
sample_submission['순위'] = sample_submission['순위'].astype(int)

# '순위' 컬럼에 0이 포함된 행들에 201부터 1800까지의 숫자를 넣어줌
mask = sample_submission['순위'] == 0
sample_submission.loc[mask, '순위'] = range(201, 1801)


sample_submission.to_csv('/content/drive/MyDrive/Colab Notebooks/0724_winsize1다시_featureimportance001_columnsdrop_pluscolumns_durationplus.csv', index=False)

# 결과 확인
sample_submission

,종목코드,순위
0,A000020,201
1,A000040,202
2,A000050,1813
3,A000070,150
4,A000080,203
...,...,...
1995,A375500,87
1996,A378850,1798
1997,A383220,1846
1998,A383310,1799
